# Import 

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# Explore

In [ ]:
df_meta = pd.read_csv('/projects/cpjump1/jump/metadata/complete_metadata.csv', low_memory=False)

In [ ]:
df_meta_mol = df_meta[df_meta['Metadata_PlateType'] == 'COMPOUND']
df_mol = df_meta_mol[['Metadata_JCP2022', 'Metadata_InChI', 'Metadata_InChIKey']].drop_duplicates(inplace=False).reset_index(drop=True)

In [ ]:
df_mol

In [ ]:
df_chembl = pd.read_csv('/projects/synsight/repos/phenoseeker/data/ChEMBL/chembl_activity_data.csv', low_memory=False)
df_assays = pd.read_csv('/projects/synsight/repos/phenoseeker/data/ChEMBL/assay.csv')

In [ ]:
df_assays

In [ ]:
len(df_chembl.columns)

In [ ]:
assay_index = df_chembl.columns

In [ ]:
allowed_assays = [int(x) for x in assay_index if x.isdigit()]

# Supposons que votre DataFrame s'appelle df et qu'elle possède une colonne 'assay_id'
filtered_df = df_assays[df_assays['assay_id'].isin(allowed_assays)]



In [ ]:
df_assays['assay_category'].value_counts()

In [ ]:
df_chembl

In [ ]:
cols = df_chembl.columns.difference(['molregno', 'standard_inchi_key'])

# Compter le nombre de 1 par colonne
count_1 = (df_chembl[cols] == 1).sum()

# Compter le nombre de -1 par colonne
count_neg1 = (df_chembl[cols] == -1).sum()

# Créer un DataFrame récapitulatif
summary_df = pd.DataFrame({
    'count_1': count_1,
    'count_-1': count_neg1
})

In [ ]:
summary_df['hit rate'] = summary_df['count_1'] / (summary_df['count_1'] + summary_df['count_-1'])   *100

In [ ]:
len(summary_df[summary_df['hit rate']<40]) + 16

In [ ]:
summary_df['hit rate'].hist(bins=100)

In [ ]:
len(colonnes_assays)

In [ ]:
import pandas as pd

# On suppose que df_chembl, df_mol et df_assay sont déjà chargés

# 1. Fusionner df_chembl et df_mol via la clé standard_inchi_key / Metadata_InChIKey
df_merged = df_chembl.merge(
    df_mol,
    how="left",
    left_on="standard_inchi_key",
    right_on="Metadata_InChIKey"
)

# 2. On détermine les colonnes d'assays à traiter (on exclut molregno et standard_inchi_key)
colonnes_assays = [
    col for col in df_chembl.columns
    if col not in ["molregno", "standard_inchi_key"]
]

# 3. Pour chaque assay, on filtre les molécules avec valeur non nulle et on vérifie le ratio actives/inactives
for assay_col in colonnes_assays:
    # Récupérer le chembl_id correspondant dans df_assay
    try:
        assay_int = int(assay_col)
    except ValueError:
        # Si la colonne ne peut pas être convertie en entier, on passe à la suivante
        continue

    mask_assay = (df_assays["assay_id"] == assay_int)
    if mask_assay.sum() == 0:
        # Si aucun assay correspondant n'est trouvé, passer au suivant
        continue

    chembl_id_value = df_assays.loc[mask_assay, "chembl_id"].values[0]

    # Filtrer pour ne conserver que les molécules dont la valeur dans cet assay est différente de 0
    df_filtered = df_merged[df_merged[assay_col] != 0].copy()
    
    # Si aucun enregistrement n'est présent, on passe à l'assay suivant
    if df_filtered.empty:
        continue

    # Calculer le nombre d'actives (valeur 1) et d'inactives (valeur -1)
    active_count = (df_filtered[assay_col] == 1).sum()
    inactive_count = (df_filtered[assay_col] == -1).sum()

    # On ne garde l'assay que si le nombre d'inactifs est non nul et le ratio actives/inactives est <= 40
    if inactive_count == 0 or (active_count / (inactive_count + active_count)) >= 0.40:
        print(f"Assay {assay_col} (chembl_id: {chembl_id_value}) ignoré (ratio {active_count}/{inactive_count} > 0.40 ou pas d'inactifs)")
        continue

    # Créer le DataFrame final avec les colonnes demandées
    df_result = df_filtered[["Metadata_JCP2022", "Metadata_InChIKey", "Metadata_InChI"]].copy()
    df_result["role_val"] = df_filtered[assay_col].map({1: "hit", -1: "none"})

    # Sauvegarder dans un CSV nommé selon le chembl_id
    output_path = f"/projects/synsight/repos/phenoseeker/data/ChEMBL/{chembl_id_value}_val.csv"

    df_result.to_csv(output_path, index=False)
    
    print(f"Fichier sauvegardé : {output_path}")
